In [1]:
%matplotlib ipympl
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
map_points = pickle.load(open("../map_points.pkl", "rb"))
kf_visible_map_points = pickle.load(open("../kf_visible_map_points.pkl", "rb"))
kf_poses = pickle.load(open("../kf_poses.pkl", "rb"))

In [3]:
import numpy as np
import numpy.linalg as la

eps = 0.00001

def svd(A):
    u, s, vh = la.svd(A)
    S = np.zeros(A.shape)
    S[:s.shape[0], :s.shape[0]] = np.diag(s)
    return u, S, vh

#def inverse_sigma(S):
#    inv_S = S.copy().transpose()
#    for i in range(min(S.shape)):
#        if abs(inv_S[i, i]) > eps :
#            inv_S[i, i] = 1.0/inv_S[i, i]
#    return inv_S

#def svd_solve(A, b):
#    U, S, Vt = svd(A)
#    inv_S = inverse_sigma(S)
#    svd_solution = Vt.transpose() @ inv_S @ U.transpose() @ b
#    
#    print('U:')
#    print(U)
#    print('Sigma:')
#    print(S)
#    print('V_transpose:')
#    print(Vt)
#    print('--------------')
#    print('SVD solution:')
#    print(svd_solution)
#    print('A multiplies SVD solution:')
#    print(A @ svd_solution)
#
#    return svd_solution

#############################################

def fit_plane_LSE(points):
    # points: Nx4 homogeneous 3d points
    # return: 1d array of four elements [a, b, c, d] of
    # ax+by+cz+d = 0
    assert points.shape[0] >= 3 # at least 3 points needed
    U, S, Vt = svd(points)
    null_space = Vt[-1, :]
    return null_space

def get_point_dist(points, plane):
    # return: 1d array of size N (number of points)
    dists = np.abs(points @ plane) / np.sqrt(plane[0]**2 + plane[1]**2 + plane[2]**2)
    return dists

def fit_plane_LSE_RANSAC(points, iters=1000, inlier_thresh=0.05, return_outlier_list=False):
    # points: Nx4 homogeneous 3d points
    # return: 
    #   plane: 1d array of four elements [a, b, c, d] of ax+by+cz+d = 0
    #   inlier_list: 1d array of size N of inlier points
    max_inlier_num = -1
    max_inlier_list = None
    
    N = points.shape[0]
    assert N >= 3

    for i in range(iters):
        chose_id = np.random.choice(N, 3, replace=False)
        chose_points = points[chose_id, :]
        tmp_plane = fit_plane_LSE(chose_points)
        
        dists = get_point_dist(points, tmp_plane)
        tmp_inlier_list = np.where(dists < inlier_thresh)[0]
        tmp_inliers = points[tmp_inlier_list, :]
        num_inliers = tmp_inliers.shape[0]
        if num_inliers > max_inlier_num:
            max_inlier_num = num_inliers
            max_inlier_list = tmp_inlier_list
        
        #print('iter %d, %d inliers' % (i, max_inlier_num))

    final_points = points[max_inlier_list, :]
    plane = fit_plane_LSE(final_points)
    
    fit_variance = np.var(get_point_dist(final_points, plane))
    print('RANSAC fit variance: %f' % fit_variance)
    print(plane)

    dists = get_point_dist(points, plane)

    select_thresh = inlier_thresh * 1

    inlier_list = np.where(dists < select_thresh)[0]
    if not return_outlier_list:
        return plane, inlier_list
    else:
        outlier_list = np.where(dists >= select_thresh)[0]
        return plane, inlier_list, outlier_list

In [4]:
map_points_h = cv2.convertPointsToHomogeneous(map_points).reshape(-1, 4)

In [5]:
plane, inlier_list = fit_plane_LSE_RANSAC(map_points_h, iters=1000, inlier_thresh=1, return_outlier_list=False)
print(inlier_list.shape)

RANSAC fit variance: 0.049652
[-1.40588749e-04  2.26389812e-04 -5.04696927e-02  9.98725557e-01]
(28460,)


In [6]:
# project all map points onto the plane
def plane_to_hessian(plane):
    """Convert plane to Hessian normal form (n.x + p = 0)"""
    a, b, c, d = plane
    nn = np.sqrt(a**2+b**2+c**2)
    n = np.array([a/nn, b/nn, c/nn])
    p = d/nn
    return n, p

def project_points(plane, points):
    """Project points with shape (-1, 3) onto plane (given as coefficients a, b, c, d with ax+by+cz+d=0)."""
    n, p = plane_to_hessian(plane)
    p_orig = -n*p
    points_proj = points.reshape(-1, 3) - (np.sum(n*(points.reshape(-1, 3) - p_orig.reshape(1, 3)), axis=1)*n.reshape(3, 1)).T
    return points_proj

In [7]:
map_points_proj = project_points(plane, map_points[inlier_list])

In [8]:
map_points_proj

array([[ -8.23593863,  -0.72612257,  19.80830471],
       [-15.15437065,  -3.60306988,  19.81467175],
       [  4.93939079,   7.37889237,  19.80795975],
       ...,
       [ 13.64901245,   6.10554681,  19.77798637],
       [  0.04279526,   6.05080549,  19.8156424 ],
       [  2.51661845,  -0.69291404,  19.77850127]])

In [17]:
# transform points from world to plane coordinates
def get_world2plane_transformation(plane, points, return_base=True):
    """Yields an affine transformation M from 3D world to 2D plane coordinates.

    Args:
    
        plane (`numpy.ndarray`): Shape (4,). Plane coefficients [a, b, c, d] which fullfill
            ax + by + cz + d = 0.

        points (`numpy.ndarray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.
        
        return_base (`bool`): Wether to return the orthonormal base or not.
    
    Returns:
    
        M (`numpy.ndarray`): Shape (4, 4). Affine transformation matrix which maps points on the plane
        from 3D world coordinates to 2D points with respect to a randomly chosen orthonormal base on the plane.
        Compute point2D = M @ point3D to map a 3D point to 2D coordinates. To retrieve a 3D point from 2D 
        planes coordinates compute point3D = inv(M) @ point2D.
        
        base (`numpy.ndarray`) Shape (3, 4). right-handend orthonormal base in which 2D plane points are 
        expressed. Column vectors of the array correspond to the origin point O of the base, the first and 
        second base vector u, v and the third base vector n which is the plane normal.  
    """
    # pick a random point on the plane as origin and another one to from first base vector
    point_idx = np.arange(0, 100)#points.shape[0])
    np.random.seed(0)
    plane_O, plane_U = points[np.random.choice(point_idx, 2, replace=False), :]
    u = (plane_U - plane_O)/np.linalg.norm(plane_U - plane_O)
    n, _ = plane_to_hessian(plane)  # plane normal
    # compute third base vector
    v = np.cross(u, n)/np.linalg.norm(np.cross(u, n))
    # get end points of base vectors
    U = plane_O + u
    V = plane_O + v
    N = plane_O + n
    # form base quadruplet
    D = np.array([[0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1],
                  [1, 1, 1, 1]])
    # form transformation matrix S with M * S = D
    S = np.stack((np.append(plane_O, 1), np.append(U, 1), np.append(V, 1), np.append(N, 1)), axis=1)
    # compute affine transformation M which maps points from world to plane coordinates
    M = np.matmul(D, np.linalg.inv(S))
    if return_base:
        base = np.stack([plane_O.T, u.T, v.T, n.T], axis=1)
        return M, base, plane_O, U, V, N
    else:
        return M


def map_points_world2plane(points_world, M):
    """Transforms 3D points on a plane to 2D plane coordinates given the transformation matrix M.
    
    Args:
    
        points_world (`numpy.ndarrray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.

        M (`nnumpy.ndarray`): Shape (4, 4). Affine transformation matrix computed with `get_world2plane_transformation`.
    
    Returns:
    
        points_plane (`numpy.ndarrray`): Shape (-1, 2). 2D points on the plane in (xp, yp, z=0) plane coordinates 
        w.r.t. to a randomly chosen orthonormal base.
    """
    points_world_h = cv2.convertPointsToHomogeneous(points_world).reshape(-1, 4)
    points_plane_h = (M @ points_world_h.T).T
    points_plane = cv2.convertPointsFromHomogeneous(points_plane_h).reshape(-1, 3)
    points_plane = points_plane[:, :2]
    return points_plane


def map_points_plane2world(points_plane, M):
    """Transforms 2D plane points into 3D world coordinates given the transformation matrix M.
    
    Args:
    
        points_plane (`numpy.ndarrray`): Shape (-1, 2). 2D points on the plane in (xp, yp, z=0) plane coordinates 
        w.r.t. to a randomly chosen orthonormal base.

        M (`nnumpy.ndarray`) Shape (4, 4). Affine transformation matrix computed with `get_world2plane_transformation`.
    
    Returns:
    
        points_world (`numpy.ndarrray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.
    """
    points_plane_tmp = np.zeros((points_plane.shape[0], 3))
    points_plane_tmp[:, :2] = points_plane
    points_plane_h = cv2.convertPointsToHomogeneous(points_plane_tmp).reshape(-1, 4)
    points_world_h = (np.linalg.inv(M) @ points_plane_h.T).T
    points_world = cv2.convertPointsFromHomogeneous(points_world_h).reshape(-1, 3)
    return points_world

In [18]:
# test functions
points_world = map_points_proj
print(points_world)
M, base, plane_O, U, V, N = get_world2plane_transformation(plane, points_world)
points_plane = map_points_world2plane(points_world, M)
print(points_plane)
#points_world = map_points_plane2world(points_plane, M)
#print(points_world)

[[ -8.23593863  -0.72612257  19.80830471]
 [-15.15437065  -3.60306988  19.81467175]
 [  4.93939079   7.37889237  19.80795975]
 ...
 [ 13.64901245   6.10554681  19.77798637]
 [  0.04279526   6.05080549  19.8156424 ]
 [  2.51661845  -0.69291404  19.77850127]]
[[20.42317923 -2.6266948 ]
 [27.85412305 -3.58723734]
 [ 4.96092596 -3.07312093]
 ...
 [-1.95042845  2.37784201]
 [ 9.86679842 -4.36654156]
 [11.0894213   2.71188531]]


In [14]:
base

array([[ 1.07720378e+01, -8.66507956e-01,  4.99155490e-01,
        -2.78556857e-03],
       [ 7.19235610e+00, -4.99163232e-01, -8.66496363e-01,
         4.48559609e-03],
       [ 1.97908755e+01,  1.74675117e-04, -5.27725811e-03,
        -9.99986060e-01]])

In [614]:
# method to map from image plane to estimated groud plane (GP)
# 1) for each KF, select visible points from map_points
# 2) get four map_points from subset with smallest distance from GP
# 3) project these points onto the GP
# 4) use cv2.getPerspectiveTransform() to compute a homography between the four selected and projected GP points and their correpsonding image points in the KF
# 5) project KF image onto GP by using cv2.warpPerspective()

# alternatives:
# 2b) get N>4 (e.g. N=10) map_points with smallest distance from GP
# 3b) project points onto GP
# 4b) use cv2.findHomography() to estimate the best fit homography via RANSAC

In [40]:
fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_world[:1000, 0], points_world[:1000, 1], points_world[:1000, 2], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")
ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([0,40])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")

ax.scatter(0, 0, 0, c="black", s=5)
plt.show() 

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [38]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.scatter(points_world[:1000, 0], points_world[:1000, 1], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")
ax.set_xlim([-20,30])
ax.set_ylim([-20,20])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect(1.0)
ax.scatter(base[0, 0], base[1, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], c="green") 
#ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], c="blue")

#ax.scatter(plane_O[0], plane_O[1], c="orange")
#ax.scatter(U[0], U[1], c="red")
#ax.scatter(V[0], V[1], c="green") 
#ax.scatter(N[0], N[1], c="blue")
ax.scatter(0, 0, c="black", s=5)

# base [[Ox, ux, vx, nx],
#       [Oy, uy, vy, ny],
#       [Oz, uz, vz, nz]]

ax.grid()
plt.show() 

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()